## import packages

In [31]:
import os
from datetime import datetime
import pandas as pd

from pandas import DataFrame
from typing import List

from dotenv import load_dotenv

load_dotenv()

True

## Variables

### Env

In [ ]:
INPUT_FOLDER: str = os.getenv('INPUT_FOLDER_PATH')

### other

In [19]:
time_columns: list = ['System', 'Queue', 'Ring', 'Talk', 'Hold', 'ACW', 'Consult', 'Disposition', 'Duration']
vdn_flow_columns: list = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
call_work_codes_columns: list = ['1.1', '2.1', '3.1', '4.1', '5.1']

## Functions

In [ ]:
def summarize_unused_columns(dataset_column_index: pd.core.indexes.base.Index) -> List[str]:
   """
   
   """
   unused_column_names: List[str] = [col_name for col_name in dataset_column_index if 'unnamed' in str(col_name).lower()]
   return unused_column_names

In [15]:
def filter_unused_columns(dataset: DataFrame) -> DataFrame:
    """
    
    """
    input_dataset: DataFrame = dataset.copy()
    unused_column_names: List[str] = summarize_unused_columns(dataset.columns)
    clean_df: DataFrame = input_dataset.drop(labels=unused_column_names, axis='columns')
    return clean_df

## core logic

In [7]:
test_df: DataFrame = pd.read_excel(os.path.join(INPUT_FOLDER, 'Week 1 -Mei.xlsx'), header=1)
test_df.head()

,Unnamed: 0,CallId,Segment,Start,Stop,Calling Party,Trunk Group,Dialed Number,Answered Agent,Original Agent,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,1.1,2.1,3.1,4.1,5.1
0,NaN,582539,1,2025-05-06 08:00:33,2025-05-06 08:12:07,2807 (Astrid Nederlof),NaN,8008 (SGD Serviceloket),95120 (Laurens Brand),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,582524,1,2025-05-06 08:01:02,2025-05-06 08:05:02,0620485126,101 (Vodafone01),8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,582547,1,2025-05-06 08:04:04,2025-05-06 08:16:12,7114 (Prya Bhola),NaN,8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,582561,1,2025-05-06 08:04:46,2025-05-06 08:22:13,NaN,NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,582544,1,2025-05-06 08:05:10,2025-05-06 08:14:30,0639621473,101 (Vodafone01),8008 (SGD Serviceloket),95120 (Laurens Brand),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
test_df = filter_unused_columns(dataset=test_df)
test_df.columns

Index([          'CallId',          'Segment',            'Start',
                   'Stop',    'Calling Party',      'Trunk Group',
          'Dialed Number',   'Answered Agent',   'Original Agent',
       'Call Disposition',           'Vector',         'Priority',
                  'Split',            'Level',           'System',
                  'Queue',             'Ring',             'Talk',
                   'Hold',              'ACW',          'Consult',
            'Disposition',         'Duration',                  1,
                        2,                  3,                  4,
                        5,                  6,                  7,
                        8,                  9,              '1.1',
                    '2.1',              '3.1',              '4.1',
                    '5.1'],
      dtype='object')

In [32]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-06 08:00:33 <class 'str'>


In [33]:
def set_datetime_dtype_values(dataset: DataFrame, columns: List[str]) -> DataFrame:
    for col_name in columns:
        dataset[col_name] = dataset[col_name].map(lambda str_value: datetime.strptime(str_value, "%Y-%m-%d %H:%M:%S"))

    return dataset

In [34]:
test_df = set_datetime_dtype_values(test_df, columns=['Start'])

In [35]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-06 08:00:33 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
